In [5]:
import pandas as pd

In [68]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import csv
import json
import re
from tqdm import tqdm
from openai import AsyncClient
import asyncio

In [67]:
load_dotenv()

assert os.getenv("OPENAI_API_KEY") 

openai_client = AsyncClient()

In [8]:
def csv_converter(csv_file, jsonl_file):
    with open(csv_file, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        with open(jsonl_file, 'w', encoding='utf-8') as jsonl_file:
            for row in csv_reader:
                json_line = json.dumps(row, ensure_ascii=False)
                jsonl_file.write(json_line + '\n')

csv_converter("data.csv", "converted_data")

In [ ]:
async def generate_response_cot(messages, data_json):

    response = await openai_client.chat.completions.create(
    model="o4-mini-2025-04-16",
    messages = messages,
    response_format= {"type": "json_object"},
    )

    try:
        parsed_response_content = json.loads(response.choices[0].message.content)
    except TypeError as e:
        return False, None, None
    output_value = parsed_response_content.get("answer")
    cot_ans = parsed_response_content.get("cot")
    
    if (output_value == data_json["original_key"]) and (output_value == data_json["validator_answer"]):
        return True, output_value, cot_ans
    else:
        return False, output_value, cot_ans


In [93]:
async def check_answer_cot(background_info):
    out_answer = []
    cot_answers = []
    binary_answer = []
    choice_a = ""
    choice_b = ""
    pattern = r'"(.*?)"'

    with open(r"C:\Users\tolly\Desktop\MARS\nyu-debate-modeling\task_relevant\converted_data", "r", encoding="utf-8") as f:
        for line in tqdm(f, desc="Processing"):
            data_json = json.loads(line.strip())

            choice_a = data_json.get("choice_a")
            choice_b = data_json.get("choice_b")
            sentence = data_json.get("sentence")


            # lines = data_json["prompt"].splitlines()

            # for l in lines:
            #     if l.startswith("(A)"):
            #         choice_a = l
            #     elif l.startswith("(B)"):
            #         choice_b = l
    
            # matches = re.findall(pattern, data_json["prompt"])

            prompt = f"""
            Consider the English sentence "{sentence}" 
            Which of the following is a better translation of this sentence into standard Lojban? 
            {choice_a}  
            {choice_b} 
            Exactly one of the answers above is correct. Please provide your response in JSON format, including an 'answer' field for the final result and a 'cot' (chain of thought) field explaining your reasoning.

            Example JSON format:
            {{
                "answer": "Your final answer here",
                "cot": "Your step-by-step reasoning process here."
            }}
            """

            prompt_1 = f"""
            Consider the English sentence "{sentence}" 
            Which of the following is a better translation of this sentence into standard Lojban? 
            {choice_a}  
            {choice_b} 
            Exactly one of the answers above is correct. Please provide your response in JSON format, including an 'answer' (A or B) field for the final result and a 'cot' (chain of thought) field explaining your reasoning.

            Example JSON format:
            {{
                "answer": "Your final answer here",
                "cot": "Your step-by-step reasoning process here."
            }}

            Please use the background information provided below to help you answer given question:
            {background_info}
            """
    
            messages=[
                {"role": "system", "content": "You are an expert Lojban speaker"},
                {"role": "user", "content": f"{prompt_1}"},
            ]
            # b_ans, output_model, cot_model = generate_response_cot(messages, data_json)
            # print(generate_response_cot(messages, data_json))
            b_ans, out_ans, cot_ans = await generate_response_cot(messages, data_json)
            # correct_answers.append(output_model)
            cot_answers.append(cot_ans)
            binary_answer.append(b_ans)
            out_answer.append(out_ans) 
            
    return binary_answer, out_answer, cot_answers


In [38]:
with open("background_test.txt", "r", encoding="utf-8") as f:
    background_content = f.read()

In [76]:
final_binary, final_out, final_cot = check_answer_cot(background_content)

Processing: 0it [00:00, ?it/s]


RuntimeError: asyncio.run() cannot be called from a running event loop

In [40]:
df = pd.DataFrame({
    "Binary Answers": final_binary,
    "Final Answers": final_out,
    "CoT Answers": final_cot
})

In [47]:
print(final_cot[0])

In Lojban there are two common predicates for “think that …” and “believe that …”: mi pensi lo du‘u and mi jinvi lo du‘u. mi pensi lo du‘u expresses a tentative thought or opinion (“I think that…”), whereas mi jinvi lo du‘u is stronger—“I hold as true that…,” equivalent to English “I believe that.”

In English “I think we need to hurry up” uses “think” as a hedge or opinion marker, not a firm statement of belief. Therefore the Lojban equivalent should use pensi, not jinvi. Thus the better translation is:

mi pensi lo du‘u mi’a nitcu lo ka sutryze’a

The other choice (“mi jinvi lo du‘u …”) would mean “I believe that we need to hurry,” which is a stronger assertion than the English sentence.


In [65]:
for val, indx in zip((df[df["Binary Answers"] == False]["CoT Answers"].to_list()), (df.index[df["Binary Answers"] == False].to_list())):
    print(f"\nCoT Answer for jbo_{indx + 1}:")
    print(val)


CoT Answer for jbo_1:
In Lojban there are two common predicates for “think that …” and “believe that …”: mi pensi lo du‘u and mi jinvi lo du‘u. mi pensi lo du‘u expresses a tentative thought or opinion (“I think that…”), whereas mi jinvi lo du‘u is stronger—“I hold as true that…,” equivalent to English “I believe that.”

In English “I think we need to hurry up” uses “think” as a hedge or opinion marker, not a firm statement of belief. Therefore the Lojban equivalent should use pensi, not jinvi. Thus the better translation is:

mi pensi lo du‘u mi’a nitcu lo ka sutryze’a

The other choice (“mi jinvi lo du‘u …”) would mean “I believe that we need to hurry,” which is a stronger assertion than the English sentence.

CoT Answer for jbo_3:
We need to see what finti and fi do.   

finti   = x1 gives/makes x2 to x3    (without payment)  
se finti   swaps x1↔x2 so that x1=what was made  
fi        tags the next sumti as filling x3  

(A) lo pixra cu se finti lo ka jai spaji lo vitke  
    → se

In [52]:
for indx, val in enumerate(final_cot):
    print(f"\nCoT Answer {indx + 1}:")
    print(val)


CoT Answer 1:
In Lojban there are two common predicates for “think that …” and “believe that …”: mi pensi lo du‘u and mi jinvi lo du‘u. mi pensi lo du‘u expresses a tentative thought or opinion (“I think that…”), whereas mi jinvi lo du‘u is stronger—“I hold as true that…,” equivalent to English “I believe that.”

In English “I think we need to hurry up” uses “think” as a hedge or opinion marker, not a firm statement of belief. Therefore the Lojban equivalent should use pensi, not jinvi. Thus the better translation is:

mi pensi lo du‘u mi’a nitcu lo ka sutryze’a

The other choice (“mi jinvi lo du‘u …”) would mean “I believe that we need to hurry,” which is a stronger assertion than the English sentence.

CoT Answer 2:
1. The English jufra ‘Did you drink apple tea?’ is a yes/no question about drinking a particular beverage (‘apple tea’).
2. In Lojban the basic bridi for drinking is “do pinxe X” + question tag “xu”.
3. We must express ‘apple tea’ as a selbri + sumti or as a tanru. The w

In [43]:
sum(final_binary)/len(final_binary)

0.6209677419354839

In [ ]:
true_answer_cot = sum(final_binary)
true_answer_cot/len(final_answer_cot)

0.4032258064516129

In [ ]:
final_binary_non_cot, final_non_cot = check_answers()
sum(final_binary_non_cot)/len(final_binary_non_cot)

0.4838709677419355

In [ ]:
with open("background_test.txt", "r", encoding="utf-8") as f:
    background_content = f.read()


check_answer_cot(background_content)